In [23]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **For preparing archive**

## Download a few test scenes via GCloud using downloadS2GCloud

Need to have...
- Separated into L1C prepare vs L2A prepare
- If L1C: checked availability on GCLoud 

In [24]:
import glob
import shutil
from datetime import datetime
import pandas as pd
import time

from utils.prepS2 import *
from utils.yamlUtils import *


In [3]:
non_cogs_dir = "../S2_Download/"
cogs_dir = "../S2_ARD/"
sen2cor_8 = "~/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"
sen2cor_5 = "~/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"

s3_bucket = "public-eo-data"
s3_dir = "catapult/sentinel2/"
aws_creds = pd.read_csv("../aws_creds.csv")
access_key = aws_creds.AWSAccessKeyId.values[0]
secret_key = aws_creds.AWSSecretKey.values[0]

In [4]:
des_scenes_list = [
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE",
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KWF_20180821T013410.SAFE",
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KXE_20180821T013410.SAFE",
    "S2B_MSIL1C_20190608T221949_N0207_R029_T60KXE_20190608T233246.SAFE",
    "S2B_MSIL1C_20190608T221949_N0207_R029_T60KXF_20190608T233246.SAFE"
]

In [ ]:
#lg = 

for des_scene in des_scenes_list:
    
    # shorten scene name
    scene_name = des_scene[:-21]
    scene_name = scene_name[:-17] + scene_name.split('_')[-1]
    print ( 'Scene name: {}'.format(scene_name) )
    
    # find uuid for download via esa hub
    s2id = find_s2_uuid(des_scene)

    down_dir = non_cogs_dir + des_scene + '/'
    print ( 'Download scene dir: {}'.format(down_dir) )
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    #download_s2_granule_gcloud(des_scene, non_cogs_dir)
    download_extract_s2_esa(s2id, non_cogs_dir, down_dir) # using esa due to gcloud metadata issues + sedas confidence
    
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    
    # process to l2a
    l2a_dir = down_dir.replace('_MSIL1C', '_MSIL2A')
    print ( 'L2A temp dir: {}'.format(l2a_dir) )
    if not os.path.exists(l2a_dir):
        sen2cor_correction(sen2cor_8, down_dir, non_cogs_dir) # May need to include try of version 2.5 before certain date - to be determined...
    
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

    # convert to cog
    cog_dir = cogs_dir + scene_name.replace('_MSIL1C', '_MSIL2A') + '/'
    print ( 'COG dir: {}'.format(cog_dir) )
    conv_s2scene_cogs(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))
    
    copy_s2_metadata(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))
    
    create_yaml(cog_dir, 's2')
    
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
        
    

Scene name: S2A_MSIL1C_20180820T223011_T60KWE
Download scene dir: ../S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
ESA scene already extracted: ../S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
L2A temp dir: ../S2_Download/S2A_MSIL2A_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
COG dir: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_AOT_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B02_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B03_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B04_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2

Downloading:   3%|▎         | 23.1M/744M [00:25<11:50, 1.01MB/s]

In [ ]:

for d_id, original_scene_dir, scene_name in zip(res.uuid.values, res.filename.values, res.title):
    # print ( d_id, original_scene_dir, scene_name )
    
    n+=1
    
    #print (scene_dir)
    
    tileid = original_scene_dir.split('_')[-2]
    
    # shorten scene name - wayyyy too long...
    scene_name = scene_name[:26] + '_' + tileid
    print ( '#### SCENE {} {} of {} ####'.format(scene_name, n, res.shape[0]) )
    
    # create full path for original and cog scenes
    original_scene_dir = non_cog_dir + original_scene_dir + '/'
    scene_dir = cog_dir + scene_name + '/'
    print ( 'Target original scene dir: {}'.format(original_scene_dir) )
    print ( 'Tarket scene dir: {}'.format(scene_dir) )
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    
    download_extract_s2(d_id, non_cog_dir, original_scene_dir)
 
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

    l2a_dir = original_scene_dir.replace('_MSIL1C', '_MSIL2A')[:-39]
    
    # see if l2a prod exists to avoid running sen2cor unless needed
    l2a_dir = glob.glob(original_scene_dir.replace('_MSIL1C', '_MSIL2A')[:-39]+'*')[0]+'/'
    print ('pre', l2a_dir )
    
    if not os.path.exists(l2a_dir):
    
        sen2cor_correction(sen2cor, original_scene_dir, non_cog_dir)
        # shutil.rmtree(original_scene_dir)

        l2a_dir = glob.glob(original_scene_dir.replace('_MSIL1C', '_MSIL2A')[:-39]+'*')[0] +'/'
        print ('re-defining original to l2a: {}'.format(l2a_dir) )
    
    original_scene_dir = l2a_dir
    
    # create (or clear) cog scene directory
    if not os.path.exists(scene_dir):
        print ( 'Creating scene cog directory: {}'.format(scene_dir) )
        os.mkdir(scene_dir)
    else:
        print ( 'Scene cog directory already exists so passing: {}'.format(scene_dir) )
    
    copy_s2_metadata(original_scene_dir, scene_dir, scene_name)

    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    
    conv_s2scene_cogs(original_scene_dir, scene_dir, scene_name)

    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

    create_yaml(scene_dir, 's2')
    
    s3_upload_cogs(glob.glob(scene_dir + '*'), s3_bucket, s3_dir)
    
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    
    print ( 'Nuking original directory: {}'.format(original_scene_dir) )
    # shutil.rmtree(original_scene_dir)
    
    print ( 'Nuking intermediary scene cog dir: {}'.format(scene_dir) )
    
    print ( ' #### COMPLETED #### '.format(scene_name) )


In [ ]:
n = 0
# eventual MASTER S2 L2A function requiring esa query df and working directoy
for d_id, original_scene_dir, scene_name in zip(res.uuid.values, res.filename.values, res.title):
    # print ( d_id, original_scene_dir, scene_name )
    
    n+=1
    
    #print (scene_dir)
    
    tileid = original_scene_dir.split('_')[-2]
    
    # shorten scene name - wayyyy too long...
    scene_name = scene_name[:26] + '_' + tileid
    print ( '#### SCENE {} {} of {} ####'.format(scene_name, n, res.shape[0]) )
    
    # create full path for original and cog scenes
    original_scene_dir = non_cog_dir + original_scene_dir + '/'
    scene_dir = cog_dir + scene_name + '/'
    print ( 'Target original scene dir: {}'.format(original_scene_dir) )
    print ( 'Tarket scene dir: {}'.format(scene_dir) )
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    
    download_extract_s2(d_id, non_cog_dir, original_scene_dir)
 
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

    # create (or clear) cog scene directory
    if not os.path.exists(scene_dir):
        print ( 'Creating scene cog directory: {}'.format(scene_dir) )
        os.mkdir(scene_dir)
    else:
        print ( 'Scene cog directory already exists so passing: {}'.format(scene_dir) )
    
    copy_s2_metadata(original_scene_dir, scene_dir, scene_name)

    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    
    conv_s2scene_cogs(original_scene_dir, scene_dir, scene_name)

    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

    create_yaml(scene_dir, 's2')
    
    # s3_upload_cogs(glob.glob(scene_dir + '*'), s3_bucket, s3_dir)
    
    print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
    
    print ( 'Nuking original directory: {}'.format(original_scene_dir) )
    # shutil.rmtree(original_scene_dir)
    
    print ( 'Nuking intermediary scene cog dir: {}'.format(scene_dir) )
    
    print ( ' #### COMPLETED #### '.format(scene_name) )
    
    
    

## Try using hackathon prepare functions

## **For preparing routine**